In [ ]:
# === FISH DETECTION - COMPLETE RECOVERY ===

# 1. Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# 2. Install ultralytics
!pip install ultralytics

# 3. Import
import shutil
from pathlib import Path
from ultralytics import YOLO

# 4. Find latest backup automatically
backup_folders = list(Path("/content/drive/MyDrive").glob("fish_detection_backup_*"))
if backup_folders:
    latest_backup = sorted(backup_folders)[-1]  # Most recent
    print(f"📦 Restoring from: {latest_backup.name}")

    # 5. Restore files
    shutil.copytree(str(latest_backup), "/content", dirs_exist_ok=True)

    # 6. Load model
    model = YOLO('/content/runs/detect/train/weights/best.pt')

    print("✅ Recovery complete! Model ready for new videos.")
else:
    print("❌ No backup found in Google Drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📦 Restoring from: fish_detection_backup_20251009_105607
✅ Recovery complete! Model ready for new videos.


In [ ]:
from ultralytics import YOLO

# Load the model you already trained
model = YOLO('/content/runs/detect/train/weights/best.pt')  # This is the .pt file from your training

In [ ]:
import cv2
import os
import glob
import shutil
from google.colab import drive

# ==================== SETUP ====================
drive.mount('/content/drive')
print("✓ Google Drive mounted")

# ==================== BACKUP INPUT FILES TO DRIVE ====================
print("📦 BACKING UP ALL INPUT VIDEO FILES TO GOOGLE DRIVE...")

input_backup_path = '/content/drive/MyDrive/colab_project/input_files'
os.makedirs(input_backup_path, exist_ok=True)

# Search EVERYWHERE for videos
search_locations = [
    '/content',
    '/content/drive/MyDrive',
    '/content/drive/MyDrive/colab_project',
    '/content/drive/MyDrive/colab_project/input_files',
    '/content/sample_data'
]

video_extensions = ['.mp4', '.MP4', '.avi', '.mov', '.MOV', '.mkv', '.webm']
all_video_paths = []
backed_up_count = 0

for location in search_locations:
    if os.path.exists(location):
        print(f"🔍 Scanning: {location}")
        for root, dirs, files in os.walk(location):
            for file in files:
                if any(file.lower().endswith(ext.lower()) for ext in video_extensions):
                    full_path = os.path.join(root, file)
                    all_video_paths.append(full_path)

                    # Backup input file to Google Drive
                    backup_dest = os.path.join(input_backup_path, file)
                    if not os.path.exists(backup_dest):
                        try:
                            shutil.copy2(full_path, backup_dest)
                            backed_up_count += 1
                            if backed_up_count % 50 == 0:
                                print(f"   Backed up {backed_up_count} input files...")
                        except Exception as e:
                            print(f"❌ Failed to backup {file}: {e}")

print(f"✅ INPUT BACKUP COMPLETE: {backed_up_count} files backed up to {input_backup_path}")

# Remove duplicates and sort
all_video_paths = sorted(list(set(all_video_paths)))

print(f"\n🎯 FOUND {len(all_video_paths)} VIDEOS TOTAL")

if len(all_video_paths) < 934:
    print("❌ WARNING: Still missing some videos!")
else:
    print("✅ SUCCESS: Found all 934+ videos!")

# Show sample files to verify
print("\n📹 Sample files (first 10):")
for i, path in enumerate(all_video_paths[:10]):
    print(f"  {i+1:2d}. {os.path.basename(path)}")

# ==================== FIND EXISTING RESULTS ====================
print("\n🔍 CHECKING EXISTING RESULTS...")

results_dir = "/content/results"
os.makedirs(results_dir, exist_ok=True)

# Check what result files already exist locally
existing_local_results = []
if os.path.exists(results_dir):
    for file in os.listdir(results_dir):
        if file.endswith('_fish_detailed_sizes.txt'):
            existing_local_results.append(file)

print(f"📁 Local result files: {len(existing_local_results)}")

# Check Google Drive backup (for output files)
output_backup_path = '/content/drive/MyDrive/colab_project/output_files'
existing_backup_results = []
if os.path.exists(output_backup_path):
    for file in os.listdir(output_backup_path):
        if file.endswith('_fish_detailed_sizes.txt'):
            existing_backup_results.append(file)

print(f"📁 Google Drive backup files: {len(existing_backup_results)}")

# ==================== IDENTIFY VIDEOS THAT NEED PROCESSING ====================
print("\n🔍 IDENTIFYING VIDEOS THAT NEED PROCESSING...")

# Combine all existing results (local + backup)
all_existing_results = set(existing_local_results + existing_backup_results)
print(f"📊 Total unique result files found: {len(all_existing_results)}")

# Find which videos don't have result files yet
videos_to_process = []
for video_path in all_video_paths:
    video_filename = os.path.basename(video_path)
    video_basename = os.path.splitext(video_filename)[0]
    expected_result_file = f"{video_basename}_fish_detailed_sizes.txt"

    if expected_result_file not in all_existing_results:
        videos_to_process.append(video_path)

print(f"\n📊 PROCESSING PLAN:")
print(f"📹 Total videos found: {len(all_video_paths)}")
print(f"✅ Already processed: {len(all_existing_results)}")
print(f"🔄 Need processing: {len(videos_to_process)}")
print(f"🎯 Target: 934 videos")

# Show which videos will be processed
if videos_to_process:
    print(f"\n📹 Videos to process (first 10):")
    for i, video in enumerate(videos_to_process[:10]):
        print(f"  {i+1:2d}. {os.path.basename(video)}")

# ==================== VALIDATE VIDEOS ====================
print(f"\n🔍 VALIDATING VIDEOS...")

def is_video_valid(video_path):
    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            return False
        ret, frame = cap.read()
        cap.release()
        return ret and frame is not None
    except:
        return False

valid_videos_to_process = []
corrupted_videos = []

for video_path in videos_to_process:
    if is_video_valid(video_path):
        valid_videos_to_process.append(video_path)
    else:
        corrupted_videos.append(video_path)

print(f"📊 VALIDATION RESULTS:")
print(f"✅ Valid videos to process: {len(valid_videos_to_process)}")
print(f"❌ Corrupted videos: {len(corrupted_videos)}")

# ==================== PROCESSING FUNCTION WITH IMMEDIATE BACKUP ====================
def process_video_with_detailed_info(video_path, output_file):
    try:
        print(f"🔍 Analyzing: {os.path.basename(video_path)}")
        results = model.predict(video_path, stream=True, conf=0.5)

        with open(output_file, 'w') as f:
            f.write("frame,center_x,center_y,width_px,height_px\n")

            detection_count = 0
            for frame_idx, r in enumerate(results):
                boxes = r.boxes
                if boxes is not None:
                    for box in boxes:
                        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                        width_px = x2 - x1
                        height_px = y2 - y1
                        center_x = (x1 + x2) / 2
                        center_y = (y1 + y2) / 2
                        f.write(f"{frame_idx},{center_x:.2f},{center_y:.2f},{width_px:.2f},{height_px:.2f}\n")
                        detection_count += 1

        print(f"✅ Complete: {detection_count} detections -> {output_file}")

        # ========== IMMEDIATE OUTPUT BACKUP ==========
        backup_path = os.path.join(output_backup_path, os.path.basename(output_file))
        shutil.copy2(output_file, backup_path)
        print(f"📦 IMMEDIATELY BACKED UP TO DRIVE: {backup_path}")
        # =============================================

        return True

    except Exception as e:
        print(f"❌ ERROR: {os.path.basename(video_path)} - {str(e)}")
        return False

# Verify model
print("\n🤖 MODEL STATUS:")
print("Model loaded successfully!")
print(f"Model classes: {model.names}")

# ==================== PROCESS THE VIDEOS ====================
if valid_videos_to_process:
    print(f"\n🎬 STARTING PROCESSING OF {len(valid_videos_to_process)} VIDEOS...")
    print("💡 Each file will be IMMEDIATELY backed up to Google Drive after processing")

    successful_count = 0
    failed_count = 0

    for i, video_path in enumerate(valid_videos_to_process, 1):
        print(f"\n{'='*60}")
        print(f"Processing [{i}/{len(valid_videos_to_process)}]: {os.path.basename(video_path)}")
        print(f"{'='*60}")

        base_name = os.path.splitext(os.path.basename(video_path))[0]
        output_file = os.path.join(results_dir, f"{base_name}_fish_detailed_sizes.txt")

        success = process_video_with_detailed_info(video_path, output_file)

        if success:
            successful_count += 1

            # Show preview
            print(f"\nFirst 3 lines:")
            with open(output_file, 'r') as f:
                for j, line in enumerate(f):
                    if j < 4:
                        print(line.strip())

            # Show stats
            with open(output_file, 'r') as f:
                lines = f.readlines()
                detection_count = len([line for line in lines if not line.startswith('#') and 'frame' not in line])
                print(f"Total detections: {detection_count}")
        else:
            failed_count += 1

        # Progress update
        if i % 10 == 0:
            print(f"\n📈 Progress: {i}/{len(valid_videos_to_process)}")
            print(f"   ✅ Successful: {successful_count} | ❌ Failed: {failed_count}")

    print(f"\n🎉 PROCESSING COMPLETED!")
    print(f"   ✅ Successful: {successful_count}")
    print(f"   ❌ Failed: {failed_count}")

else:
    print(f"\n💡 No videos need processing!")

# ==================== FINAL SUMMARY ====================
print(f"\n{'='*60}")
print("📊 FINAL SUMMARY")
print(f"{'='*60}")

total_processed_now = len(valid_videos_to_process)
total_all_results = len(all_existing_results) + successful_count

print(f"📹 Total videos discovered: {len(all_video_paths)}")
print(f"📦 Input files backed up: {backed_up_count}")
print(f"📄 Total result files (existing + new): {total_all_results}")
print(f"🎯 Progress: {total_all_results}/934 ({total_all_results/934*100:.1f}%)")

if total_all_results >= 934:
    print("🎉 CONGRATULATIONS! All 934 videos processed!")
elif total_all_results > 0:
    print(f"🔄 Remaining: {934 - total_all_results} videos")
else:
    print("❌ No videos were processed")

print(f"\n📁 INPUT BACKUP: {input_backup_path}")
print(f"📁 OUTPUT BACKUP: {output_backup_path}")
print(f"🚀 Script completed!")

Streaming output truncated to the last 5000 lines.
video 1/1 (frame 2203/3509) /content/drive/MyDrive/colab_project/input_files/hour_11__2025-06-19__192.168.8.101.mp4: 480x640 1 fish, 9.7ms
video 1/1 (frame 2204/3509) /content/drive/MyDrive/colab_project/input_files/hour_11__2025-06-19__192.168.8.101.mp4: 480x640 1 fish, 9.6ms
video 1/1 (frame 2205/3509) /content/drive/MyDrive/colab_project/input_files/hour_11__2025-06-19__192.168.8.101.mp4: 480x640 (no detections), 10.4ms
video 1/1 (frame 2206/3509) /content/drive/MyDrive/colab_project/input_files/hour_11__2025-06-19__192.168.8.101.mp4: 480x640 1 fish, 9.7ms
video 1/1 (frame 2207/3509) /content/drive/MyDrive/colab_project/input_files/hour_11__2025-06-19__192.168.8.101.mp4: 480x640 1 fish, 10.0ms
video 1/1 (frame 2208/3509) /content/drive/MyDrive/colab_project/input_files/hour_11__2025-06-19__192.168.8.101.mp4: 480x640 1 fish, 9.4ms
video 1/1 (frame 2209/3509) /content/drive/MyDrive/colab_project/input_files/hour_11__2025-06-19__192.16